In [1]:
import torch
import numpy as np
pwd = '/home/zjy/project/IM'
device = torch.device('cuda:5' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:5


In [2]:
import networkx as nx

network_index = -1
seed_index = 0
seed_rates = [0.01,0.05,0.10,0.20,0.30]
seed_rate = seed_rates[seed_index]
NetWorks = ['deezer_europe','email-Enron','ca-GrQc','Celegans','cora','ego-Facebook','fb-pages-food','Lastfm_asia','musae-github','roadNet-CA','Router','Soc-hamsterster', 'USA airports','Yeast','wiki-Talk','wiki-Vote','p2p-Gnutella05','soc-Slashdot0811','soc-dolphins','congress-twitter','bitcoin-alpha','bitcoin-otc','cit-HepPh','soc-douban']
direct = False
network = NetWorks[network_index]
print('Reading {} edgelist'.format(network))
network_edges_dir = pwd+'/data/{}/{}.txt'.format(network,network)
G = None
if direct:
    with open(network_edges_dir, 'rb')as edges_f:
        G = nx.read_edgelist(edges_f, nodetype=int, create_using=nx.DiGraph(), encoding='latin1',
                                    data=(('weight', float),))
else:
    with open(network_edges_dir, 'rb')as edges_f:
        G = nx.read_edgelist(edges_f, nodetype=int, create_using=nx.Graph(), encoding='latin1',
                                    data=(('weight', float),))

Reading soc-douban edgelist


In [9]:
sum(dict(G.degree()).values()) / len(G),G.number_of_nodes(), G.number_of_edges()

(4.223965192243138, 154908, 327163)

In [398]:
from torch_geometric.nn import MessagePassing

import torch.nn.functional as F

class SIR(MessagePassing):
    def __init__(self, N, beta, gamma, C ,steps, device):
        super().__init__(aggr="add")
        self.N = N
        self.beta = beta
        self.gamma = gamma
        self.C = C
        self.steps = steps
        self.device = device

    def init_state(self,x0=None):
        if x0 is None:
            x = torch.zeros(self.N,1).to(self.device)
            ind = torch.argsort(self.w.view(-1),descending=True)
            x[ind[:self.C]] = 1    
        else:
            x = x0    
        r = torch.zeros(self.N,1).to(self.device)
        s = 1 - x
        return s, x ,r
    
    def single_step_forward(self, edge_index, s, x, r):
        q = self.propagate(edge_index, x=torch.log(1-self.beta*x))
        q = torch.exp(q)
        s, x, r = s*q, (1-self.gamma)*x + s*(1-q) , r + self.gamma*x
        return s, x, r

    def forward(self, edge_index, x0=None):
        s, x, r = self.init_state(x0)
        for i in range(self.steps): 
           s, x ,r = self.single_step_forward(edge_index,s,x,r)
        return s, x, r

    def message(self, x_j):
        return x_j

In [399]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class IC(nn.Module):
    def __init__(self, N, steps, device, C=50):
        super(IC, self).__init__()
        self.steps = steps
        self.N = N
        self.C = C
        self.device = device
        self.gamma = 1.0
        
    def init_state(self,x0=None):
        if x0 is None:
            x = torch.zeros(self.N,1).to(self.device)
            ind = torch.argsort(self.w.view(-1),descending=True)
            x[ind[:self.C]] = 1    
        else:
            x = x0    
        r = torch.zeros(self.N,1).to(self.device)
        s = 1 - x
        return s, x ,r
    
    def single_step_forward(self, edge_index, edge_weight, s, x, r):
        # 进行加权聚合
        row, col = edge_index
        # ww = (torch.ones(edge_weight.shape)*self.beta).to(self.device)
        epsilon = 1e-15
        agg_messages = torch.log(1- (edge_weight * x[row])+ epsilon)  # 加权消息传递 (num_edges, hidden_dim)
        agg_messages = torch.zeros_like(x).scatter_add_(0, col.unsqueeze(-1), agg_messages)
        q = torch.exp(agg_messages)
        s, x, r = s*q, s*(1-q) , r + x
        return s, x, r
    
    def forward(self, edge_index, edge_weight, x0=None):
        s, x, r = self.init_state(x0)
        for i in range(self.steps): 
           s, x ,r = self.single_step_forward(edge_index,edge_weight,s,x,r)
        return s, x, r
    def message(self, x_j):
        return x_j

In [400]:
def infected_beta(graph):
    # SIR模型的感染率设置
    degree = nx.degree(graph)
    degree_list = []
    degree_sq_list = []
    for i in degree:
        degree_list.append(i[1])
        degree_sq_list.append(i[1] * i[1])
    degree_avg = np.mean(degree_list)
    degree_avg_sq = np.mean(degree_sq_list)
    infected = degree_avg / (degree_avg_sq -  2 *degree_avg)
    return infected
N = G.number_of_nodes()
# gamma = 1  # 免疫率
# beta = 2 *infected_beta(G) * gamma
steps = 10
seed_num = int(N* seed_rate)

# sir = SIR(N,beta,gamma,50,steps,device).to(device)
ic = IC(N, steps,device).to(device)

In [404]:
edge_index = []
edge_weight = []
if direct:
    # 添加所有边
    for edge in G.edges():
        edge_index.append([edge[0], edge[1]])
        edge_weight.append(1/G.in_degree[edge[1]])
else:
    # 添加所有边
    for edge in G.edges():
        edge_index.append([edge[0], edge[1]])
        edge_weight.append(1/G.degree[edge[1]])
        
        edge_index.append([edge[1], edge[0]])  # 如果是无向图，添加反向边
        edge_weight.append(1/G.degree[edge[0]])

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous().to(device)
edge_weight = torch.tensor(edge_weight, dtype=torch.float).unsqueeze(-1).to(device)

In [405]:
def get_influence(x_hat, seed_num, edge_index):
    _, top_seeds_predict = torch.topk(x_hat, seed_num, sorted=False)
    list_pre = top_seeds_predict.tolist()
    x0 = torch.zeros(N,1).to(device)
    x0[list_pre] = 1
    # _,_,r = sir(edge_index, x0=x0)
    _,_,r = ic(edge_index,edge_weight, x0=x0)
    infection = r.squeeze().cpu()
    Influence_num = infection.sum()
    return Influence_num


In [409]:
class BinaryActivationSTE(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        # Sigmoid + Threshold 0.5
        return (x > 0.95).float()
    
    @staticmethod
    def backward(ctx, grad_output):
        # 直通估计：忽略非连续性，直接传梯度
        return grad_output
    

In [410]:
from torch.optim import SGD,Adam,RMSprop,Adadelta,Adagrad
import torch.nn.init as init

y_true = torch.ones(N).to(device)
x_init = 0.9 + 0.1 *torch.rand(N).to(device)

x_init.requires_grad = True
initial_momentum = 0.95
z_optimizer = SGD([x_init], lr=0.01, momentum=initial_momentum)

max_x = None
max_influence_num = 0
now_influence_num = 0
lam = 10
for i in range(5000):
    z_optimizer.zero_grad()
    # x = BinaryActivationSTE.apply(torch.sigmoid(x_init))
    x = BinaryActivationSTE.apply(x_init)
    # x = x_init
    seeds_t = x.unsqueeze(-1)
    
    # seeds_t = x_init.unsqueeze(-1)
    # seeds_t = F.sigmoid(seeds_t)
    
    _, _, y_hat = ic(edge_index,edge_weight, x0=seeds_t)
    # _, _, y_hat = sir(edge_index, x0=seeds_t)

    # l0 = (seeds_t*(1-seeds_t)).mean()
    # loss = F.mse_loss(y_true, y_hat.squeeze(-1))
    # loss = F.mse_loss(y_true, y_hat.squeeze(-1)) + lam*torch.abs(seeds_t.mean() - seed_rate)
    loss = -y_hat.squeeze(-1).mean() + lam*torch.abs(seeds_t.mean() - seed_rate)
    # loss = - y_hat.mean() + 0.5*(x_init*(1-x_init)).mean()
    # loss = - y_hat.mean()
    
    loss.backward()
    # normalized_grad = x_init.grad / torch.norm(x_init.grad,p=p)
    # x_init.grad.copy_(normalized_grad)
    # x_grad_sum = x_init.grad.sum()
    # grad_l2_norm = torch.norm(x_init.grad)
    # 检查梯度是否存在NaN
    # normalized_grad = -torch.abs(x_init.grad)
    # x_init.grad.copy_(normalized_grad)
    # 检查梯度是否存在NaN
    
    z_optimizer.step()
    

    now_influence_num = get_influence(x_init, seed_num, edge_index)
    if now_influence_num > max_influence_num:
        max_influence_num = now_influence_num
        max_x = x_init.clone().detach()

    print('Iteration: {}'.format(i+1),
        '\t Loss:{:.5f}'.format(loss.item()),
        "\t Influence_num: {:.4f}".format(now_influence_num),
        "\t X_init_sum: {:.4f}".format(seeds_t.sum()),
        # "\t l0: {:.4f}".format(l0.item()),
        # "\t Momentum: {:.4f}".format(new_momentum),
        )

Iteration: 1 	 Loss:4.07583 	 Influence_num: 660.4893 	 X_init_sum: 3526.0000
Iteration: 2 	 Loss:4.07175 	 Influence_num: 660.4893 	 X_init_sum: 3523.0000
Iteration: 3 	 Loss:4.07175 	 Influence_num: 660.4893 	 X_init_sum: 3523.0000
Iteration: 4 	 Loss:4.06764 	 Influence_num: 660.4893 	 X_init_sum: 3520.0000
Iteration: 5 	 Loss:4.06217 	 Influence_num: 660.4893 	 X_init_sum: 3516.0000
Iteration: 6 	 Loss:4.05550 	 Influence_num: 660.4893 	 X_init_sum: 3511.0000
Iteration: 7 	 Loss:4.05145 	 Influence_num: 660.4893 	 X_init_sum: 3508.0000
Iteration: 8 	 Loss:4.04915 	 Influence_num: 660.4893 	 X_init_sum: 3506.0000
Iteration: 9 	 Loss:4.04223 	 Influence_num: 660.4893 	 X_init_sum: 3501.0000
Iteration: 10 	 Loss:4.03569 	 Influence_num: 660.4893 	 X_init_sum: 3496.0000
Iteration: 11 	 Loss:4.03163 	 Influence_num: 660.4893 	 X_init_sum: 3493.0000
Iteration: 12 	 Loss:4.01957 	 Influence_num: 660.4893 	 X_init_sum: 3484.0000
Iteration: 13 	 Loss:4.01293 	 Influence_num: 667.3224 	 X_in

KeyboardInterrupt: 

In [212]:
for i in range(10000):
    z_optimizer.zero_grad()
    # x = BinaryActivationSTE.apply(torch.sigmoid(x_init))
    x = BinaryActivationSTE.apply(x_init)
    seeds_t = x.unsqueeze(-1)
    
    # seeds_t = x_init.unsqueeze(-1)
    # seeds_t = F.sigmoid(seeds_t)
    
    _, _, y_hat = ic(edge_index,edge_weight, x0=seeds_t)
    # _, _, y_hat = sir(edge_index, x0=seeds_t)

    # loss = F.mse_loss(y_true, y_hat.squeeze(-1))
    loss = -y_hat.squeeze(-1).mean() + lam*torch.abs(seeds_t.mean() - seed_rate)
    # loss = - y_hat.mean() + 0.5*(x_init*(1-x_init)).mean()
    # loss = - y_hat.mean()
    
    loss.backward()
    # normalized_grad = x_init.grad / torch.norm(x_init.grad,p=p)
    # x_init.grad.copy_(normalized_grad)
    # x_grad_sum = x_init.grad.sum()
    # grad_l2_norm = torch.norm(x_init.grad)
    # 检查梯度是否存在NaN
    # normalized_grad = -torch.abs(x_init.grad)
    # x_init.grad.copy_(normalized_grad)
    # 检查梯度是否存在NaN
    
    z_optimizer.step()
    
    # decay_rate = 0.05  # 衰减速率
    # new_momentum = initial_momentum - (initial_momentum * decay_rate * (i / 100))
    # if new_momentum < 0:  
    #     new_momentum = 0
    # for param_group in z_optimizer.param_groups:
    #     param_group['momentum'] = new_momentum
    
        
    
    # with torch.no_grad():
    #     x_init.clamp_(min=0.0, max=1.0)
    #     x_init.copy_(l1_projection(x_init, seed_num))
        # x_init.copy_(lp_projection(x_init,p, norm_t))

    now_influence_num = get_influence(x_init, seed_num, edge_index)
    if now_influence_num > max_influence_num:
        max_influence_num = now_influence_num
        max_x = x_init.clone().detach()

    print('Iteration: {}'.format(i+1),
        '\t Loss:{:.5f}'.format(loss.item()),
        "\t Influence_num: {:.4f}".format(now_influence_num),
        "\t X_init_sum: {:.4f}".format(seeds_t.sum()),
        # "\t Momentum: {:.4f}".format(new_momentum),
        )

Iteration: 1 	 Loss:-0.75521 	 Influence_num: 5339.1650 	 X_init_sum: 707.0000
Iteration: 2 	 Loss:-0.75521 	 Influence_num: 5339.1650 	 X_init_sum: 707.0000
Iteration: 3 	 Loss:-0.75521 	 Influence_num: 5339.1650 	 X_init_sum: 707.0000
Iteration: 4 	 Loss:-0.75476 	 Influence_num: 5339.1650 	 X_init_sum: 706.0000
Iteration: 5 	 Loss:-0.75476 	 Influence_num: 5339.1650 	 X_init_sum: 706.0000
Iteration: 6 	 Loss:-0.75476 	 Influence_num: 5339.1650 	 X_init_sum: 706.0000
Iteration: 7 	 Loss:-0.75476 	 Influence_num: 5339.1650 	 X_init_sum: 706.0000
Iteration: 8 	 Loss:-0.75521 	 Influence_num: 5339.1650 	 X_init_sum: 707.0000
Iteration: 9 	 Loss:-0.75521 	 Influence_num: 5339.1650 	 X_init_sum: 707.0000
Iteration: 10 	 Loss:-0.75521 	 Influence_num: 5339.1650 	 X_init_sum: 707.0000
Iteration: 11 	 Loss:-0.75521 	 Influence_num: 5339.1650 	 X_init_sum: 707.0000
Iteration: 12 	 Loss:-0.75521 	 Influence_num: 5339.1650 	 X_init_sum: 707.0000
Iteration: 13 	 Loss:-0.75476 	 Influence_num: 53

KeyboardInterrupt: 

In [411]:
import sys
sys.path.append(pwd+'/utils')
import graph_utils as graph_utils

inf_num = get_influence(max_x, seed_num, edge_index)
print(inf_num)
_, top_seeds_predict = torch.topk(max_x, seed_num, sorted=False)
list_pre = top_seeds_predict.tolist()
graph, N = graph_utils.read_graph(network_edges_dir, ind=0, directed=direct)
time = 1000
# inf = graph_utils.workerMC([graph,list_pre,time])
# ,graph_utils.computeMC_IC(graph,data_list,time)/N
num = graph_utils.computeMC_IC(graph,list_pre,time).sum()
num/N

tensor(3456.3843)


0.5114431078403623